In [22]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout, Flatten, Reshape, multiply
from keras.layers import Input, Dense, LSTM, concatenate, Activation, GRU, SimpleRNN, Masking
import numpy as np

In [120]:
f = np.load("mfcc_fixed.npz")
X, Y = f['X'], f['Y']
X_train =  X[0:900]
Y_train =  Y[0:900]      # fold 10


x_test, y_test = X[900:1000], Y[900:1000]
x_train, y_train = X_train[0:800], Y_train[0:800]
x_val, y_val= X_train[800:900], Y_train[800:900]

In [121]:
batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)

In [122]:
def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor

    Returns: a keras tensor
    '''
    filters = input.shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se


def build_model(input_shape, batch_size, num_classes):
    inputs = Input(shape=input_shape, batch_size=batch_size)     # input_shape=(len, dim)
    ip = Permute((2, 1))(inputs)  # input_shape=(dim, len)

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, out)
    # model.summary()

    # add load model code here to fine-tune

    return model


In [123]:
model = build_model(X[0].shape, batch_size, 5)

In [124]:
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(100, 54, 30)]              0         []                            
                                                                                                  
 permute_26 (Permute)        (100, 30, 54)                0         ['input_14[0][0]']            
                                                                                                  
 permute_27 (Permute)        (100, 54, 30)                0         ['permute_26[0][0]']          
                                                                                                  
 conv1d_39 (Conv1D)          (100, 54, 128)               30848     ['permute_27[0][0]']          
                                                                                           

In [125]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
acc_metric = keras.metrics.SparseCategoricalAccuracy()
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

def scheduler(epoch, lr):
    if lr > 1e-4:
        return lr * np.floor(epoch/100)*(1-0.7937005259841)
    else:
        return lr
    
rLRoP_callback = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                    factor=0.7937005259841,
                                                    patience=100,
                                                    verbose=0,
                                                    mode="auto",
                                                    min_delta=0.0001,
                                                    cooldown=0,
                                                    min_lr=1e-4)
es_callback = keras.callbacks.EarlyStopping(
                                            monitor="val_loss",
                                            min_delta=0,
                                            patience=100,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True,
                                            start_from_epoch=0,
                                           )

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(train_dataset, validation_data=val_dataset, batch_size=batch_size, epochs=2000,
          callbacks=[es_callback, rLRoP_callback])

Epoch 1/2000
8/8 [==============================] - 5s 137ms/step - loss: 0.9964 - accuracy: 0.6237 - val_loss: 4.0395 - val_accuracy: 0.2700 - lr: 0.0010
Epoch 2/2000
8/8 [==============================] - 0s 13ms/step - loss: 0.3351 - accuracy: 0.9200 - val_loss: 7.7458 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.1850 - accuracy: 0.9563 - val_loss: 9.5296 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 4/2000
8/8 [==============================] - 0s 13ms/step - loss: 0.1316 - accuracy: 0.9638 - val_loss: 10.0119 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 5/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.0807 - accuracy: 0.9900 - val_loss: 9.3525 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 6/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.0676 - accuracy: 0.9812 - val_loss: 8.4579 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 7/2000
8/8 [==============================] - 0s 12ms/step - loss: 0

In [127]:
acc_metric.reset_states()
acc_metric.update_state(y_test, model(x_test))
acc = acc_metric.result().numpy()
loss = loss_fn(y_test, model(x_test)).numpy()
print(acc, loss)

1.0 6.335396e-05
